In [ ]:
!pip install transformers torch torchvision pillow langchain langchain-community langchain-text-splitters pypdf sentence-transformers faiss-cpu beautifulsoup4 requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from PIL import Image
import torch
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from datetime import datetime

class IntegratedStructuralAnalysisSystem:
    def __init__(self, pdf_path=None):
        self.pdf_path = pdf_path
        self.vectorstore = None
        self.blip_processor = None
        self.blip_model = None
        self.vqa_processor = None
        self.vqa_model = None
        self.classifier = None

        # Bangladesh Market Rates
        self.market_rates = {
            'rod_60_grade': 95,
            'rod_40_grade': 85,
            'cement_holcim': 540,
            'cement_fresh': 530,
            'stone_chips_per_cft': 85,
            'sand_per_cft': 55,
            'skilled_labor_jacketing': 1200,
            'mason_daily': 1500,
            'helper_daily': 800,
            'bonding_agent_per_liter': 450,
            'epoxy_per_kg': 850,
            'concrete_mixer_rental': 2500,
            'scaffolding_per_sqft': 45,
            'form_work_per_sqft': 180,
            'chicken_mesh_per_sqft': 65,
            'crack_injection_per_ft': 1200,
            'surface_repair_per_sqft': 350,
            'frp_wrapping_per_sqft': 3500,
            'steel_plate_per_kg': 95,
            'waterproofing_per_sqft': 180,
        }

    def setup(self):
        """Setup all models"""
        print("=" * 80)
        print("INITIALIZING INTEGRATED STRUCTURAL ANALYSIS & COSTING SYSTEM")
        print("=" * 80)

        # 1. Image Captioning Model
        print("\n[1/4] Loading image captioning model (BLIP)...")
        self.blip_processor = BlipProcessor.from_pretrained(
            "Salesforce/blip-image-captioning-large"
        )
        self.blip_model = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-image-captioning-large"
        )
        print("      ✓ BLIP model loaded")

        # 2. Visual Question Answering Model
        print("\n[2/4] Loading Visual QA model (BLIP-VQA)...")
        self.vqa_processor = BlipProcessor.from_pretrained(
            "Salesforce/blip-vqa-base"
        )
        self.vqa_model = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-vqa-base"
        )
        print("      ✓ VQA model loaded")

        # 3. Text Classification
        print("\n[3/4] Loading zero-shot classifier...")
        self.classifier = pipeline(
            "zero-shot-classification",
            model="facebook/bart-large-mnli"
        )
        print("      ✓ Classifier loaded")

        # 4. Load PDF guidelines
        if self.pdf_path:
            print("\n[4/4] Loading building guidelines PDF...")
            loader = PyPDFLoader(self.pdf_path)
            documents = loader.load()

            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )
            texts = text_splitter.split_documents(documents)

            embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )
            self.vectorstore = FAISS.from_documents(texts, embeddings)
            print(f"      ✓ Loaded {len(documents)} pages from PDF")

        print("\n" + "=" * 80)
        print("✅ SYSTEM READY!")
        print("=" * 80 + "\n")

    def analyze_image(self, image_path):
        """Generate detailed image description"""
        image = Image.open(image_path).convert('RGB')

        # General caption
        inputs = self.blip_processor(image, return_tensors="pt")
        out = self.blip_model.generate(**inputs, max_length=100)
        general_caption = self.blip_processor.decode(out[0], skip_special_tokens=True)

        # Detailed caption
        text_prompt = "a detailed view of"
        inputs = self.blip_processor(image, text_prompt, return_tensors="pt")
        out = self.blip_model.generate(**inputs, max_length=100)
        detailed_caption = self.blip_processor.decode(out[0], skip_special_tokens=True)

        return general_caption, detailed_caption

    def ask_visual_questions(self, image_path):
        """Ask specific questions about the image"""
        image = Image.open(image_path).convert('RGB')

        questions = [
            "Is this a crack?",
            "Is the crack diagonal or horizontal?",
            "Is there exposed rebar?",
            "Is there concrete spalling?",
            "Is this on a column or beam?",
            "Is the crack wide?",
            "Is this structural damage?",
            "How many cracks are visible?",
            "Is this a wall or column?"
        ]

        answers = {}
        for question in questions:
            inputs = self.vqa_processor(image, question, return_tensors="pt")
            out = self.vqa_model.generate(**inputs, max_length=20)
            answer = self.vqa_processor.decode(out[0], skip_special_tokens=True)
            answers[question] = answer

        return answers

    def classify_defect(self, description, vqa_answers):
        """Classify defect type"""
        full_description = f"{description}. "
        for q, a in vqa_answers.items():
            full_description += f"{q} {a}. "

        candidate_labels = [
            "structural shear crack",
            "structural flexural crack",
            "structural column damage",
            "concrete spalling with rebar exposure",
            "short column defect",
            "cosmetic plaster crack",
            "cosmetic paint damage",
            "cosmetic surface issue"
        ]

        result = self.classifier(
            full_description,
            candidate_labels,
            multi_label=False
        )

        top_label = result['labels'][0]
        top_score = result['scores'][0]

        is_structural = 'structural' in top_label.lower()
        classification = "STRUCTURAL" if is_structural else "COSMETIC"

        severity = "LOW"
        if is_structural:
            if any(kw in full_description.lower() for kw in ['rebar', 'spalling', 'wide', 'column']):
                severity = "CRITICAL"
            elif 'shear' in top_label or 'diagonal' in full_description.lower():
                severity = "HIGH"
            else:
                severity = "MEDIUM"

        return {
            'classification': classification,
            'predicted_defect': top_label,
            'confidence': top_score,
            'severity': severity,
            'all_predictions': list(zip(result['labels'], result['scores'])),
            'description': full_description
        }

    def calculate_repair_cost(self, classification_result, vqa_answers,
                            estimated_area_sqft=None,
                            estimated_crack_length_ft=None):
        """
        ছবি analysis এর ভিত্তিতে মেরামত খরচ হিসাব করা
        """

        defect_type = classification_result['predicted_defect']
        severity = classification_result['severity']
        is_structural = classification_result['classification'] == "STRUCTURAL"

        # Default estimates based on image analysis
        if estimated_area_sqft is None:
            # Estimate from visual answers
            if 'column' in vqa_answers.get('Is this on a column or beam?', '').lower():
                estimated_area_sqft = 100  # Typical column surface
            elif 'beam' in vqa_answers.get('Is this on a column or beam?', '').lower():
                estimated_area_sqft = 80
            else:
                estimated_area_sqft = 50  # Wall section

        if estimated_crack_length_ft is None:
            crack_count = vqa_answers.get('How many cracks are visible?', 'one')
            if 'multiple' in crack_count.lower() or 'many' in crack_count.lower():
                estimated_crack_length_ft = 15
            else:
                estimated_crack_length_ft = 8

        print("\n" + "=" * 80)
        print("💰 REPAIR COST ESTIMATION")
        print("=" * 80)
        print(f"\nDefect Type: {defect_type}")
        print(f"Severity: {severity}")
        print(f"Estimated Area: {estimated_area_sqft} sqft")
        print(f"Estimated Crack Length: {estimated_crack_length_ft} ft")

        costs = {}
        total_cost = 0

        # ==================== STRUCTURAL DEFECTS ====================

        if "shear crack" in defect_type or "flexural crack" in defect_type:
            print("\n" + "-" * 80)
            print("REPAIR METHOD: EPOXY INJECTION + CARBON FIBER WRAPPING")
            print("-" * 80)

            # 1. Crack injection
            injection_cost = estimated_crack_length_ft * self.market_rates['crack_injection_per_ft']
            costs['Epoxy Crack Injection'] = injection_cost

            # 2. Carbon fiber wrapping
            wrapping_cost = estimated_area_sqft * self.market_rates['frp_wrapping_per_sqft']
            costs['Carbon Fiber/FRP Wrapping'] = wrapping_cost

            # 3. Surface preparation
            prep_cost = estimated_area_sqft * 200
            costs['Surface Preparation'] = prep_cost

            # 4. Labor
            days = max(3, estimated_area_sqft / 30)
            labor_cost = days * (self.market_rates['mason_daily'] * 2 +
                                self.market_rates['helper_daily'] * 2)
            costs['Labor'] = labor_cost

        elif "column damage" in defect_type or "short column" in defect_type:
            print("\n" + "-" * 80)
            print("REPAIR METHOD: COLUMN JACKETING (3-INCH CONCRETE)")
            print("-" * 80)

            # Assume standard column
            perimeter = (estimated_area_sqft / 10)  # Rough estimate
            jacket_volume = perimeter * 0.25 * 10  # 3 inch thick, 10 ft height

            # Steel
            steel_kg = jacket_volume * 2.5
            steel_cost = steel_kg * self.market_rates['rod_60_grade']
            costs['Steel (60-grade)'] = steel_cost

            # Cement
            cement_bags = jacket_volume * 0.18
            cement_cost = cement_bags * self.market_rates['cement_holcim']
            costs['Cement'] = cement_cost

            # Sand & Stone
            costs['Sand'] = jacket_volume * 1.5 * self.market_rates['sand_per_cft']
            costs['Stone Chips'] = jacket_volume * 2 * self.market_rates['stone_chips_per_cft']

            # Bonding agent
            costs['Epoxy Bonding Agent'] = estimated_area_sqft * 0.15 * self.market_rates['bonding_agent_per_liter']

            # Wire mesh
            costs['Wire Mesh'] = estimated_area_sqft * self.market_rates['chicken_mesh_per_sqft']

            # Formwork
            costs['Formwork'] = estimated_area_sqft * self.market_rates['form_work_per_sqft']

            # Labor
            days = max(7, estimated_area_sqft / 20)
            costs['Skilled Labor'] = estimated_area_sqft * self.market_rates['skilled_labor_jacketing']
            costs['Mason & Helpers'] = days * (self.market_rates['mason_daily'] * 2 +
                                               self.market_rates['helper_daily'] * 3)

            # Equipment
            costs['Scaffolding'] = estimated_area_sqft * self.market_rates['scaffolding_per_sqft']
            costs['Mixer Rental'] = days * self.market_rates['concrete_mixer_rental']

        elif "spalling" in defect_type:
            print("\n" + "-" * 80)
            print("REPAIR METHOD: CONCRETE PATCHING + REBAR TREATMENT")
            print("-" * 80)

            # Rebar treatment (anti-corrosion)
            costs['Rebar Cleaning & Anti-corrosion'] = estimated_area_sqft * 500

            # Concrete patching
            patch_volume = estimated_area_sqft * 0.5  # 6 inch deep patch
            costs['Micro-concrete Patch Material'] = patch_volume * 1200

            # Bonding agent
            costs['Bonding Agent'] = estimated_area_sqft * 0.2 * self.market_rates['bonding_agent_per_liter']

            # Waterproofing
            costs['Waterproofing Coating'] = estimated_area_sqft * self.market_rates['waterproofing_per_sqft']

            # Labor
            days = max(4, estimated_area_sqft / 25)
            costs['Labor'] = days * (self.market_rates['mason_daily'] * 2 +
                                    self.market_rates['helper_daily'] * 2)

        # ==================== COSMETIC DEFECTS ====================

        elif "plaster crack" in defect_type or "paint damage" in defect_type:
            print("\n" + "-" * 80)
            print("REPAIR METHOD: SURFACE REPAIR + REPLASTERING")
            print("-" * 80)

            # Surface repair
            costs['Crack Filling & Repair'] = estimated_area_sqft * self.market_rates['surface_repair_per_sqft']

            # Replastering
            costs['Cement Plaster'] = estimated_area_sqft * 250

            # Paint
            costs['Paint (2 coats)'] = estimated_area_sqft * 120

            # Labor
            days = max(2, estimated_area_sqft / 40)
            costs['Labor'] = days * (self.market_rates['mason_daily'] +
                                    self.market_rates['helper_daily'])

        else:
            # Generic repair
            costs['General Repair'] = estimated_area_sqft * 800
            costs['Labor'] = estimated_area_sqft * 300

        # Calculate totals
        subtotal = sum(costs.values())

        # Add contingency based on severity
        if severity == "CRITICAL":
            contingency_rate = 0.20  # 20%
        elif severity == "HIGH":
            contingency_rate = 0.15  # 15%
        else:
            contingency_rate = 0.10  # 10%

        contingency = subtotal * contingency_rate
        total_cost = subtotal + contingency

        # Display breakdown
        print("\n💰 COST BREAKDOWN:")
        print("=" * 80)
        for item, cost in costs.items():
            print(f"{item:40s}: ৳ {cost:>12,.2f}")

        print("-" * 80)
        print(f"{'SUBTOTAL':40s}: ৳ {subtotal:>12,.2f}")
        print(f"{'Contingency (' + str(int(contingency_rate*100)) + '%)':40s}: ৳ {contingency:>12,.2f}")
        print("=" * 80)
        print(f"{'TOTAL ESTIMATED COST':40s}: ৳ {total_cost:>12,.2f}")
        print("=" * 80)

        # Additional recommendations
        print("\n📋 WORK BREAKDOWN:")
        print("-" * 80)

        if is_structural:
            duration_days = max(7, int(estimated_area_sqft / 15))
            print(f"Estimated Duration: {duration_days} working days")
            print(f"Engineer Supervision: Required (৳ 25,000 - 50,000)")
            print(f"Structural Testing: Required (৳ 15,000 - 30,000)")
        else:
            duration_days = max(3, int(estimated_area_sqft / 30))
            print(f"Estimated Duration: {duration_days} working days")
            print(f"Engineer Supervision: Optional")

        print(f"\n💳 PAYMENT SCHEDULE (Recommended):")
        print("-" * 80)
        print(f"Advance Payment (30%):        ৳ {total_cost * 0.30:>12,.2f}")
        print(f"Mid-stage Payment (40%):      ৳ {total_cost * 0.40:>12,.2f}")
        print(f"Final Payment (30%):          ৳ {total_cost * 0.30:>12,.2f}")

        print(f"\n⚠️ IMPORTANT NOTES:")
        print("-" * 80)
        if is_structural:
            print("• Hire RAJUK-approved contractor")
            print("• Obtain structural engineer certification")
            print("• Follow BNBC 2020 guidelines")
            print("• Minimum 28-day curing period")
            print("• Post-repair inspection mandatory")
        else:
            print("• Use quality materials for durability")
            print("• Proper surface preparation essential")
            print("• Allow adequate curing time")

        print("\n" + "=" * 80 + "\n")

        return {
            'total_cost': total_cost,
            'subtotal': subtotal,
            'contingency': contingency,
            'breakdown': costs,
            'duration_days': duration_days,
            'estimated_area': estimated_area_sqft
        }

    def analyze_and_estimate(self, image_path, area_sqft=None, crack_length_ft=None):
        """
        সম্পূর্ণ analysis: ছবি দেখা + defect classify + cost estimate
        """

        print("\n" + "#" * 80)
        print("# COMPLETE STRUCTURAL ANALYSIS & COST ESTIMATION")
        print("#" * 80)
        print(f"Image: {image_path}")
        print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("#" * 80)

        # Step 1: Image Analysis
        print("\n" + "=" * 80)
        print("STEP 1: IMAGE ANALYSIS")
        print("=" * 80)

        print("\n🔍 Analyzing image...")
        general, detailed = self.analyze_image(image_path)
        print(f"   General Description: {general}")
        print(f"   Detailed Description: {detailed}")

        # Step 2: Visual Q&A
        print("\n🔍 Visual Question Answering...")
        vqa_answers = self.ask_visual_questions(image_path)
        for q, a in vqa_answers.items():
            print(f"   Q: {q}")
            print(f"   A: {a}")

        # Step 3: Classification
        print("\n" + "=" * 80)
        print("STEP 2: DEFECT CLASSIFICATION")
        print("=" * 80)

        classification = self.classify_defect(detailed, vqa_answers)

        print(f"\n🏷️  CLASSIFICATION: {classification['classification']}")
        print(f"⚠️  SEVERITY LEVEL: {classification['severity']}")
        print(f"🎯 PREDICTED DEFECT: {classification['predicted_defect']}")
        print(f"📈 CONFIDENCE: {classification['confidence']:.2%}")

        print("\n📋 ALL PREDICTIONS:")
        print("-" * 80)
        for label, score in classification['all_predictions'][:5]:
            indicator = "🔴" if "structural" in label.lower() else "🟢"
            print(f"{indicator} {score:>6.2%} - {label}")

        # Step 4: Recommendations
        print("\n" + "=" * 80)
        print("STEP 3: IMMEDIATE RECOMMENDATIONS")
        print("=" * 80)

        if classification['classification'] == "STRUCTURAL":
            if classification['severity'] == "CRITICAL":
                print("\n🚨 URGENT ACTION REQUIRED:")
                print("   • Immediate structural engineer inspection")
                print("   • Restrict access to affected area")
                print("   • Document and monitor closely")
                print("   • Consider temporary shoring if needed")
            elif classification['severity'] == "HIGH":
                print("\n⚠️  PRIORITY REPAIR NEEDED:")
                print("   • Schedule structural engineer inspection within 48 hours")
                print("   • Monitor for progression")
                print("   • Avoid additional loading")
            else:
                print("\n⚠️  INSPECTION RECOMMENDED:")
                print("   • Schedule structural assessment")
                print("   • Plan for repair within 1-2 months")
        else:
            print("\n✅ COSMETIC ISSUE - LOW PRIORITY:")
            print("   • Schedule routine maintenance")
            print("   • Cosmetic repair sufficient")
            print("   • No immediate safety concern")

        # Step 5: Building Code References
        if self.vectorstore:
            print("\n" + "=" * 80)
            print("STEP 4: BUILDING CODE REFERENCES (BNBC 2020)")
            print("=" * 80)

            search_query = f"{classification['predicted_defect']} repair requirements"
            docs = self.vectorstore.similarity_search(search_query, k=2)

            for i, doc in enumerate(docs, 1):
                print(f"\n[Reference {i} - Page {doc.metadata.get('page', 'N/A')}]")
                print("-" * 80)
                print(doc.page_content[:400] + "...")

        # Step 6: Cost Estimation
        print("\n" + "=" * 80)
        print("STEP 5: COST ESTIMATION")
        print("=" * 80)

        cost_result = self.calculate_repair_cost(
            classification,
            vqa_answers,
            estimated_area_sqft=area_sqft,
            estimated_crack_length_ft=crack_length_ft
        )

        # Final Summary
        print("\n" + "#" * 80)
        print("# SUMMARY REPORT")
        print("#" * 80)
        print(f"\nDefect Type:        {classification['predicted_defect']}")
        print(f"Classification:     {classification['classification']}")
        print(f"Severity:           {classification['severity']}")
        print(f"Confidence:         {classification['confidence']:.1%}")
        print(f"\nEstimated Cost:     ৳ {cost_result['total_cost']:,.2f}")
        print(f"Repair Duration:    {cost_result['duration_days']} days")
        print(f"Priority:           {'URGENT' if classification['severity'] in ['CRITICAL', 'HIGH'] else 'ROUTINE'}")
        print("#" * 80 + "\n")

        return {
            'classification': classification,
            'cost_estimation': cost_result,
            'vqa_answers': vqa_answers,
            'description': detailed
        }

# ============================================================================
# USAGE EXAMPLES
# ============================================================================

# Initialize system
system = IntegratedStructuralAnalysisSystem(pdf_path=None)  # Add PDF path if available
system.setup()

# Example 1: Analyze crack photo
print("\n" + "=" * 80)
print("EXAMPLE 1: ANALYZING STRUCTURAL CRACK")
print("=" * 80)

# Upload your image first, then run:
# result1 = system.analyze_and_estimate(
#     image_path="/content/crack_photo.jpg",
#     area_sqft=100,  # Optional: specify if known
#     crack_length_ft=10  # Optional: specify if known
# )

# Example 2: Column damage analysis
# result2 = system.analyze_and_estimate(
#     image_path="/content/column_damage.jpg",
#     area_sqft=120
# )

# Example 3: Cosmetic crack
# result3 = system.analyze_and_estimate(
#     image_path="/content/wall_crack.jpg"
# )

print("\n✅ System ready! Upload your images and run analyze_and_estimate()")

INITIALIZING INTEGRATED STRUCTURAL ANALYSIS & COSTING SYSTEM

[1/4] Loading image captioning model (BLIP)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

The image processor of type `BlipImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

      ✓ Classifier loaded

✅ SYSTEM READY!


EXAMPLE 1: ANALYZING STRUCTURAL CRACK

✅ System ready! Upload your images and run analyze_and_estimate()


In [ ]:
# 1. Upload your image in Colab
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# 2. Run complete analysis with cost estimation
result = system.analyze_and_estimate(
    image_path=image_path,
    area_sqft=100,  # Optional - system will estimate if not provided
    crack_length_ft=8  # Optional - system will estimate if not provided
)

# 3. Access results
print(f"Total Cost: ৳ {result['cost_estimation']['total_cost']:,.2f}")
print(f"Defect Type: {result['classification']['predicted_defect']}")

Saving col ceack 3.jfif to col ceack 3.jfif

################################################################################
# COMPLETE STRUCTURAL ANALYSIS & COST ESTIMATION
################################################################################
Image: col ceack 3.jfif
Analysis Date: 2026-02-05 20:57:45
################################################################################

STEP 1: IMAGE ANALYSIS

🔍 Analyzing image...
   General Description: arafed concrete column with a hole in it with a sign on it
   Detailed Description: a detailed view of a concrete pillar with a hole in it

🔍 Visual Question Answering...
   Q: Is this a crack?
   A: is this a crack?
   Q: Is the crack diagonal or horizontal?
   A: is the crack diagonal or horizontal?
   Q: Is there exposed rebar?
   A: is there exposed rebar?
   Q: Is there concrete spalling?
   A: is there concrete spalling?
   Q: Is this on a column or beam?
   A: is this on a column or beam?
   Q: Is the crack wide?
   A: is

In [ ]:
# 1. Upload your image in Colab
from google.colab import files
uploaded = files.upload()
image_path = list(uploaded.keys())[0]

# 2. Run complete analysis with cost estimation
result = system.analyze_and_estimate(
    image_path=image_path,
    area_sqft=100,  # Optional - system will estimate if not provided
    crack_length_ft=8  # Optional - system will estimate if not provided
)

# 3. Access results
print(f"Total Cost: ৳ {result['cost_estimation']['total_cost']:,.2f}")
print(f"Defect Type: {result['classification']['predicted_defect']}")

Saving Beam crack 02.jfif to Beam crack 02.jfif

################################################################################
# COMPLETE STRUCTURAL ANALYSIS & COST ESTIMATION
################################################################################
Image: Beam crack 02.jfif
Analysis Date: 2026-02-05 20:59:04
################################################################################

STEP 1: IMAGE ANALYSIS

🔍 Analyzing image...
   General Description: there is a picture of a room with a ceiling that has been stripped
   Detailed Description: a detailed view of a ceiling with a hole in the ceiling

🔍 Visual Question Answering...
   Q: Is this a crack?
   A: is this a crack?
   Q: Is the crack diagonal or horizontal?
   A: is the crack diagonal or horizontal?
   Q: Is there exposed rebar?
   A: is there exposed rebar?
   Q: Is there concrete spalling?
   A: is there concrete spalling?
   Q: Is this on a column or beam?
   A: is this on a column or beam?
   Q: Is the crack